<a href="https://colab.research.google.com/github/mart-anthony-stark/Machine-Learning-and-Deep-Learning/blob/main/Fake_News_Detection_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle

# Data Loading

In [68]:
true_df = pd.read_csv('True.csv', engine='python')
fake_df = pd.read_csv('Fake.csv', engine='python')

In [69]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [70]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [71]:
print("True data shape:", true_df.shape)
print("Fake data shape:", fake_df.shape)

True data shape: (21417, 4)
Fake data shape: (23481, 4)


In [72]:
true_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [73]:
fake_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [74]:
true_df['target'] = 'true'
fake_df['target'] = 'fake'

In [75]:
true_df.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",true
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",true
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",true


In [76]:
fake_df.head()

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


## Combine Dataset (True, Fake)

In [77]:
df = pd.concat([true_df, fake_df]).reset_index(drop=True)
df.shape

(44898, 5)

In [78]:
df.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",true
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",true
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",true


In [79]:
df.tail()

,title,text,subject,date,target
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",fake
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",fake
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",fake
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",fake
44897,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",fake


## Shuffle dataset

In [80]:
df = shuffle(df).reset_index(drop=True)

In [81]:
df.head()

,title,text,subject,date,target
0,JUST IN: Anti-Putin Banker Claims Firm Tied To...,One of the many targets of the opposition rese...,Government News,"Nov 13, 2017",fake
1,[VIDEO] Obama Needed A Girl To Take Down Mitt ...,Because Obama s doing such a great job keeping...,Government News,"Nov 3, 2015",fake
2,UPDATE: WHY UNIV OF MICHIGAN REPLACED SCHEDULE...,HUGE NEWS: The head coach of the University of...,left-news,"Apr 8, 2015",fake
3,BRILLIANT! Why “Democratic” Socialism Doesn’t ...,This is Stephen Crowder at his best So smart a...,Government News,"Apr 30, 2016",fake
4,Emirates stops flying to Tunisia in row over b...,TUNIS (Reuters) - Dubai airline Emirates will ...,worldnews,"December 24, 2017",true


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   target   44898 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


# Data Preprocessing

Removing unnecessary columns

In [83]:
df.drop(['date', 'title'], axis=1,inplace=True)
df.head()

,text,subject,target
0,One of the many targets of the opposition rese...,Government News,fake
1,Because Obama s doing such a great job keeping...,Government News,fake
2,HUGE NEWS: The head coach of the University of...,left-news,fake
3,This is Stephen Crowder at his best So smart a...,Government News,fake
4,TUNIS (Reuters) - Dubai airline Emirates will ...,worldnews,true


## Converting text to lowercase

In [84]:
df['text'] = df['text'].apply(lambda text: text.lower())
df.head()

,text,subject,target
0,one of the many targets of the opposition rese...,Government News,fake
1,because obama s doing such a great job keeping...,Government News,fake
2,huge news: the head coach of the university of...,left-news,fake
3,this is stephen crowder at his best so smart a...,Government News,fake
4,tunis (reuters) - dubai airline emirates will ...,worldnews,true
